In [1]:
from PIL import Image
import os
from glob import glob
from sklearn.model_selection import train_test_split, GridSearchCV
from matplotlib import pyplot as plt
# import tensorflow as tf
import numpy as np
# import keras as kr


In [2]:
image_dir_path = os.path.dirname(os.path.abspath("__file__")) + '/Data(224)'
categories = os.listdir('./Data(224)')
select_cat = []
nb_classes = 0


X = []
y = []

count = 0;
for idx, c in enumerate(categories):
    dir = image_dir_path + '/' + c
    f = glob(dir+'/*.jpg')
    if len(f) >= 200:
        nb_classes += 1
        select_cat.append(dir)

In [3]:
for index, cat in enumerate(select_cat):
    files = glob(select_cat[index]+'/*.jpg')
    for i, f in enumerate(files):
            img = Image.open(f)
            img = img.convert("L")
            #img = img.resize((image_w, image_h))
            data = np.asarray(img)
            
            X.append(data)
            y.append(index)

In [4]:
X = np.array(X)
y = np.array(y)

nsamples, nx, ny = X.shape
X = X.reshape((nsamples,nx*ny))

In [5]:
X_train, X_test, y_train, Y_test = train_test_split(X, y, test_size = 0.4)
x_test, x_val, y_test, y_val = train_test_split(X_test, Y_test, test_size = 0.5)


In [6]:
# from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler()
# scaler.fit(X_train)
# X_train_sc = scaler.transform(X_train)
# x_val_sc = scaler.transform(x_val)
# x_test_sc = scaler.transform(x_test)

# print(X_train_sc.shape, y_train.shape)
# print(x_val_sc.shape, y_val.shape)
# print(x_test_sc.shape, y_test.shape)

In [8]:
# QuantitleTransformer
from sklearn.preprocessing import QuantileTransformer

scaler = QuantileTransformer()
scaler.fit(X_train)
X_train_sc = scaler.transform(X_train)
x_test_sc = scaler.transform(x_test)
x_val_sc = scaler.transform(x_val)

print(X_train_sc.shape, y_train.shape)
print(x_val_sc.shape, y_val.shape)
print(x_test_sc.shape, y_test.shape)

(2127, 50176) (2127,)
(710, 50176) (710,)
(709, 50176) (709,)


In [28]:
def fitting(X,y,C,gamma):
    
    model = SVC(kernel='rbf', probability=True, C=C, gamma=gamma)
    clf = model.fit(X_train_sc, y_train)
    
    pred_labels_tr = model.predict(X_train_sc)
    pred_labels_val = model.predict(x_val_sc)
    
    print('----- Evaluation on Validation Data -----')
    score_val = model.score(x_val_sc, y_val)
    print('Accuracy Score: ', score_val)
    
    print(classification_report(y_val, pred_labels_val))
    print('--------------------------------------------------------')
    
    print('----- Evaluation on Training Data -----')
    score_tr = model.score(X_train_sc, y_train)
    print('Accuracy Score: ', score_tr)
    
    print(classification_report(y_train, pred_labels_tr))
    print('--------------------------------------------------------')
    
    return X_train_sc, x_val_sc, y_train, y_val, clf

In [29]:
def plot_3D (X, x_val_sc, y_val, clf):
    
    mesh_size = 5
    margin = 1
    
    x_min, x_max = X.iloc[:, 0].fillna(X.mean()).min() - margin, X.iloc[:, 0].fillna(X.mean()).max() + margin
    y_min, y_max = X.iloc[:, 1].fillna(X.mean()).min() - margin, X.iloc[:, 1].fillna(X.mean()).max() + margin
    xrange = np.arange(x_min, x_max, mesh_size)
    yrange = np.arange(y_min, y_max, mesh_size)
    xx, yy = np.meshgrid(xrange, yrange)
    
    Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:,1]
    Z = Z.reshape(xx.shape)
    
    fig = px.scatter_3d(x=X_test['rating_difference'], y=X_test['turns'], z=y_test,
                       opacity=0.8, color_discrete_sequence=['black'])
    
    fig.update_layout("SVM VALIDATION", paper_bgcolor = 'white', 
                     scene = dict(xaxis=dict(backgroundcolor = 'white',
                                            color = 'black',
                                            gridcolor = '#f0f0f0'),
                                 yaxis=dict(backgroundcolor='white',
                                           color = 'black',
                                           gridcolor='#f0f0f0'),
                                 zaxis=dict(backgroundcolor = 'Lightgray',
                                           color='black',
                                           gridcolor='#f0f0f0',
                                           )))
    
    fig.update_traces(marker=dict(size=1))
    
    fig.add_traces(go.Surface(x=xrange, y=yrange, z=Z, name='SVM Prediction',
                             colorscale='RdBu', showscale=False,
                             contours = {"z":{"show":True, "start":0.2, "end":0.8,"size":0.05}}))
    
    fig.show()

In [ ]:
import pandas as pd
from sklearn.metrics import classification_report
import plotly.express as px
import plotly.graph_objects as go
from sklearn.svm import SVC



X_train_sc, x_val_sc, y_train, y_val, clf = fitting(X,y,1,'scale')

In [ ]:
plot_3D(X, x_val, y_val, clf)

In [ ]:
X_train, x_val, y_train, y_test, clf = fitting(X, y, 1, 0.1)
plot_3D(X,x_val, y_val,clf)

In [ ]:
X_train, x_val, y_train, y_test, clf = fitting(X, y, 1, 0.000001)
plot_3D(X, x_val, y_val, clf)